# Creación de un nuevo modelo, con valores actualizados

In [1]:
#Ya instaladas en el entorno virtual del Jupyter Notebook
%pip install cobra python-libsbml

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Verificación de la instalación
import libsbml
import cobra
print("libSBML version:", libsbml.getLibSBMLVersion())
print("Cobra version:", cobra.__version__)
# importar librerías
import pandas as pd
import numpy as np
from cobra.io import read_sbml_model
from cobra.flux_analysis import flux_variability_analysis

libSBML version: 52002
Cobra version: 0.29.0


In [3]:
# Ruta del archivo
PstKB = '../Models/PstKB.ori.xml'
# Crear un lector de SBML
lector = libsbml.SBMLReader()
# Leer el archivo SBML y cargarlo en un objeto SBMLDocument
documento_sbml = lector.readSBML(PstKB)
# Obtener el modelo el SBML
modelo = documento_sbml.getModel()

In [4]:
# Texto parcial o clave que estás buscando en los IDs de las reacciones
clave_metabolito = 'cpd11416'
# Lista para almacenar las reacciones que contienen el metabolito
reacciones_con_metabolito = []
# Iterar a través de cada reacción en el modelo
for i in range(modelo.getNumReactions()):
    reaccion = modelo.getReaction(i)
    # Buscar el metabolito clave en los reactantes de la reacción
    for reactante in reaccion.getListOfReactants():
        if clave_metabolito in reactante.getSpecies():
            reacciones_con_metabolito.append(reaccion.getId())
            break  # Rompe el bucle interno si se encuentra el metabolito
    # Si no se encontró en los reactantes, buscar en los productos
    if reaccion.getId() not in reacciones_con_metabolito:
        for producto in reaccion.getListOfProducts():
            if clave_metabolito in producto.getSpecies():
                reacciones_con_metabolito.append(reaccion.getId())
                break  # Rompe el bucle interno si se encuentra el metabolito
# Imprimir las reacciones encontradas que contienen el metabolito
print("Reacciones que contienen el metabolito:", clave_metabolito)
for id_reaccion in reacciones_con_metabolito:
    print(id_reaccion)

Reacciones que contienen el metabolito: cpd11416
R_bio1_biomass
R_bio2_biomass
R_DM_cpd11416_c0


In [5]:
# Buscamos la reacción por ID la de biomasa como sabemos es cpd11416[c0]
id_biomasa="R_bio1_biomass"
reaccion_biomasa= modelo.getReaction(id_biomasa)
print(reaccion_biomasa)

<Reaction R_bio1_biomass "Plant Leaf (Core) auto biomass">


In [6]:
# ID de la reacción que quieres examinar
id_reaccion = 'R_bio1_biomass'

# Obtener la reacción del modelo por su ID
reaccion = modelo.getReaction(id_reaccion)

if reaccion:
    # Inicializar las partes de la ecuación de la reacción
    ecuacion_reactantes = []
    ecuacion_productos = []

    # Iterar sobre los reactantes de la reacción
    for reactante in reaccion.getListOfReactants():
        especie = modelo.getSpecies(reactante.getSpecies())
        nombre_especie = especie.getName() if especie.getName() else reactante.getSpecies()
        coeficiente = reactante.getStoichiometry()
        ecuacion_reactantes.append(f"{coeficiente} {nombre_especie}")

    # Iterar sobre los productos de la reacción
    for producto in reaccion.getListOfProducts():
        especie = modelo.getSpecies(producto.getSpecies())
        nombre_especie = especie.getName() if especie.getName() else producto.getSpecies()
        coeficiente = producto.getStoichiometry()
        ecuacion_productos.append(f"{coeficiente} {nombre_especie}")

    # Construir la ecuación de la reacción
    ecuacion = ' + '.join(ecuacion_reactantes) + " --> " + ' + '.join(ecuacion_productos)

    # Imprimir la ecuación de la reacción
    print(f"Reacción {id_reaccion}:")
    print(ecuacion)
else:
    print(f"La reacción con ID '{id_reaccion}' no se encontró en el modelo.")

Reacción R_bio1_biomass:
0.161 L-Alanine_c0 + 0.039 L-Lactate_c0 + 0.076 Ferulate_c0 + 0.074 L-Phenylalanine_c0 + 0.172 L-Leucine_c0 + 0.024 L-Tryptophan_c0 + 0.00015 NADH_c0 + 0.0001 FAD_c0 + 0.0001 5-10-Methenyltetrahydrofolate_c0 + 0.1 L-Serine_c0 + 0.03 dATP_c0 + 30.0 H2O_c0 + 0.0001 5-10-Methylenetetrahydrofolate_c0 + 0.011 ocdca_c0 + 0.026 dCTP_c0 + 0.096 L-Lysine_c0 + 0.038 L-Histidine_c0 + 0.0757 Oxaloacetate_c0 + 0.0001 NADPH_c0 + 0.015 Oleate_c0 + 0.000975 Phytonadiol_d0 + 0.0001 FMN_c0 + 0.096 L-Threonine_c0 + 0.13 L-Glutamate_c0 + 0.11 L-Arginine_c0 + 0.15 Glycine_c0 + 0.056 L-Asparagine_c0 + 0.0001 BIOT_c0 + 0.037 L-Malate_c0 + 0.083 L-Isoleucine_c0 + 0.000543 S-Adenosyl-L-methionine_c0 + 0.049 L-Methionine_c0 + 0.013 Citrate_c0 + 0.086 cis-Aconitate_c0 + 0.000155 Plastoquinol-9_c0 + 0.0001 5-Methyltetrahydrofolate_c0 + 0.766 UDP-xylose_c0 + 0.127 L-Valine_c0 + 0.2271 beta-D-Fructose_c0 + 0.026 dGTP_c0 + 0.098 L-Aspartate_c0 + 0.000136 CoA_c0 + 1.59 alpha-D-Glucose_c0 + 0.

## Prueba para definir correctamente los ID de los metabolitos

In [7]:
# Cadena que deseas buscar en los ID de los metabolitos
cadena_busqueda = 'cpd00001'
# Lista para almacenar los ID de los metabolitos que coinciden con la búsqueda
metabolitos_coincidentes = []
# Iterar a través de todas las especies (metabolitos) en el modelo
for i in range(modelo.getNumSpecies()):
    # Obtener la especie actual
    especie = modelo.getSpecies(i)
    # Obtener el ID de la especie
    id_especie = especie.getId()
    # Verificar si el ID de la especie contiene la cadena de búsqueda
    if cadena_busqueda in id_especie:
        # Agregar el ID de la especie a la lista de coincidencias
        metabolitos_coincidentes.append(id_especie)
# Imprimir los resultados
if metabolitos_coincidentes:
    print(f"Metabolitos que contienen '{cadena_busqueda}':")
    for id_metabolito in metabolitos_coincidentes:
        print(id_metabolito)
else:
    print(f"No se encontraron metabolitos que contengan '{cadena_busqueda}'.")

Metabolitos que contienen 'cpd00001':
M_cpd00001_c0
M_cpd00001_d0
M_cpd00001_g0
M_cpd00001_v0
M_cpd00001_w0
M_cpd00001_x0
M_cpd00001_n0
M_cpd00001_m0
M_cpd00001_r0
M_cpd00001_e0


In [8]:
# ID de la reacción de interés
id_reaccion = 'R_bio1_biomass'
# Obtener la reacción específica por su ID
reaccion = modelo.getReaction(id_reaccion)
# Preparar una lista para almacenar los datos de los metabolitos y sus coeficientes
metabolitos_y_coeficientes = []
if reaccion:
    # Iterar sobre los reactantes y productos de la reacción
    for lista_metabolitos in [reaccion.getListOfReactants(), reaccion.getListOfProducts()]:
        for componente in lista_metabolitos:
            id_metabolito = componente.getSpecies()
            coeficiente = componente.getStoichiometry()
            
            # Filtrar solo aquellos metabolitos cuyo ID contiene 'cpd'
            if 'cpd' in id_metabolito:
                metabolitos_y_coeficientes.append((id_metabolito, coeficiente))
# Convertir la lista en un DataFrame para visualización como tabla
df_metabolitos = pd.DataFrame(metabolitos_y_coeficientes, columns=['ID Metabolito', 'Coeficiente'])
# Mostrar la tabla
df_metabolitos.to_csv('../Data/allSpeciesBIOMASS2.csv',index=False)
df_metabolitos

,ID Metabolito,Coeficiente
0,M_cpd00035_c0,0.161
1,M_cpd00159_c0,0.039
2,M_cpd01059_c0,0.076
3,M_cpd00066_c0,0.074
4,M_cpd00107_c0,0.172
...,...,...
68,M_cpd00012_c0,0.218
69,M_cpd00067_c0,30.000
70,M_cpd11416_c0,1.000
71,M_cpd00014_c0,0.766


### Actualización de los coeficientes en la reacción R_bio1_biomass


In [9]:
# Cargar las hojas del excel como data frame separados
file_xls= '../Data/modelData.xls'
Comparative_OFs_sheet='Comparative OFs'
Comparative_OFs_columns = ['ID','Metabolite','Botero 2018','Guevara 2023','PstKB_Ori','PstKB_cobra']
Comparative_OFs = pd.read_excel(file_xls, sheet_name=Comparative_OFs_sheet, names=Comparative_OFs_columns)

In [10]:
# Construcción del data set con los valores nuevos
Comparative_OFs = Comparative_OFs.iloc[:,[0,-1]]
# Quitar [c0] o el [d0] de ID
Comparative_OFs['ID'] = Comparative_OFs['ID'].str.replace(r'\[c0\]|\[d0\]', '', regex=True)
Comparative_OFs.to_csv('../Data/ID+PstKB_cobra.csv',index=False)
Comparative_OFs

,ID,PstKB_cobra
0,cpd00001,52.486878
1,cpd00002,50.864030
2,cpd00003,0.000300
3,cpd00004,0.000150
4,cpd00005,0.000100
...,...,...
67,cpd16443,0.037700
68,cpd16503,0.000155
69,cpd19001,0.053000
70,cpd25914,0.000100


In [11]:
# Convertir el dataframe en un diccionario para fácil accesión
DiccionarioCoeficientes = pd.Series(Comparative_OFs.PstKB_cobra.values, index=Comparative_OFs.ID).to_dict()
## Como se cargo la reaccion arriba no hay necesidad aca
if reaccion:
    # Iterar sobre los reactantes y actualizar los coeficientes
    for reactante in reaccion.getListOfReactants():
        id_especie = reactante.getSpecies()
        # Comprobar si alguna clave en el diccionario está contenida en el ID de la especie
        for key in DiccionarioCoeficientes.keys():
            if key in id_especie:
                nuevo_coeficiente = DiccionarioCoeficientes[key]
                reactante.setStoichiometry(nuevo_coeficiente)
                break  # Romper el ciclo una vez que se actualiza el coeficiente

    # Repetir para los productos si es necesario
    for producto in reaccion.getListOfProducts():
        id_especie = producto.getSpecies()
        # Comprobar si alguna clave en el diccionario está contenida en el ID de la especie
        for key in DiccionarioCoeficientes.keys():
            if key in id_especie:
                nuevo_coeficiente = DiccionarioCoeficientes[key]
                producto.setStoichiometry(nuevo_coeficiente)
                break  # Romper el ciclo una vez que se actualiza el coeficiente

    print(f"Coeficientes actualizados para la reacción '{id_reaccion}'.")
else:
    print(f"No se encontró la reacción con ID '{id_reaccion}' en el modelo.")

Coeficientes actualizados para la reacción 'R_bio1_biomass'.


In [12]:
# Guardar el nuevo modelo
UpdateModelo = '../Models/PstKB_cobra.xml'
escritor = libsbml.SBMLWriter()
escritor.writeSBML(documento_sbml, UpdateModelo)

True

## Por ultimo correr el modelo con Cobrapy, para verificar que corra

In [13]:
# Modelo actualizado
file_sbml= '../Models/PstKB_cobra.xml'
PapaModel= read_sbml_model(file_sbml)
ModeloOri = read_sbml_model(PstKB)

In [14]:
print("Función Objetivo Actual:", PapaModel.objective.expression)

Función Objetivo Actual: 1.0*bio1_biomass - 1.0*bio1_biomass_reverse_6e711


In [15]:
# Convertir las matrices de flujos en DataFrames de Pandas
df_ori = ModeloOri.optimize().fluxes.reset_index()
df_ori.columns = ['ID', 'Flujo_Ori']
df_actualizado = PapaModel.optimize().fluxes.reset_index()
df_actualizado.columns = ['ID', 'Flujo_Actualizado']
# Combinar ambos DataFrames utilizando la columna 'ID' como clave
df_combinado = pd.merge(df_ori, df_actualizado, on='ID')
# Mostrar el DataFrame combinado
df_combinado

,ID,Flujo_Ori,Flujo_Actualizado
0,rxn00001_c0,-191.417904,0.0
1,rxn00001_d0,0.000000,0.0
2,rxn00001_g0,0.000000,0.0
3,rxn00001_v0,0.000000,0.0
4,rxn00001_w0,0.000000,0.0
...,...,...,...
1080,EX_cpd00099_e0,-17.698141,0.0
1081,EX_cpd00009_e0,-29.631576,0.0
1082,EX_cpd11632_e0,-1000.000000,0.0
1083,DM_cpd02701_c0,0.008428,0.0
